In [2]:
import geopandas as gpd
import numpy as np

# ThreadPoolExecutor：用于I/O密集型任务，使用线程来实现并发。
# ProcessPoolExecutor：用于计算密集型任务，使用进程来实现并发，可以绕过GIL的限制，利用多核处理器。
from concurrent.futures import ThreadPoolExecutor, as_completed,ProcessPoolExecutor

In [15]:
from shapely.geometry import Point

In [3]:
import math

In [4]:
albers_proj = '+proj=aea +lat_1=18 +lat_2=50 +lat_0=0 +lon_0=105 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'
length = 1263
dir = "E:\\greenland_Campus\\GisDataforChina\\"

utils 读取到内存

In [5]:
def BatchRead_gpkg(gpkg_files):
    # 创建一个线程池
    length = len(gpkg_files) 
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_index = {executor.submit(read_gpkg, i,gpkg_files): i for i in range(length)}

        # 创建一个列表来按顺序存储结果
        gdfs = [None] * length

        # 等待所有任务完成，并按文件编号顺序存储结果
        for future in as_completed(future_to_index):
            index = future_to_index[future]
            result = future.result()
            if result is not None:
                gdfs[index] = result
    return gdfs

def read_gpkg(i,gpkg_files):
    try:
        gdf = gpd.read_file(gpkg_files[i])
        gdf=gdf.dropna(subset=['geometry'])
        return gdf
    except Exception as e:
        print(f"Error reading {gpkg_files[i]}: {e}")
    return None

In [ ]:
# 先运行以上所有

## 1.aoi属性： 面积area、角度orient、长宽比aspect_ratio

In [ ]:
aoi_gdf = gpd.read_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_geo.gpkg') # 这是地理坐标系中的aoi

In [ ]:
aoi_oob = gpd.read_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_geo_oob_width.gpkg') # 这是地理坐标系中的oob

计算Albers面积

In [ ]:
# 定义亚洲阿尔伯斯投影的参数
# 例如，中央经线为105度，标准纬线为18度和50度
albers_proj = '+proj=aea +lat_1=18 +lat_2=50 +lat_0=0 +lon_0=105 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

# 将数据投影到亚洲阿尔伯斯投影
gdf_albers = aoi_gdf.to_crs(albers_proj)
gdf_albers.area
aoi_gdf['面积Albers']=gdf_albers.area

### 拆分为单个要素，计算面积和形状指数

In [ ]:
# 仅拆分（仍然是地理坐标系）
for i in range(0,1263):
    range_gdf = gpd.GeoDataFrame([aoi_gdf.iloc[i]],  columns=aoi_gdf.columns,crs=aoi_gdf.crs)
    range_gdf.to_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\campus_aoi_{}.gpkg'.format(i))

In [ ]:
# 这里因为修改了aoi文件中 campus7 的位置，重新导出
i=7
aoi_gdf = gpd.read_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_geoT2.gpkg')
range_gdf = gpd.GeoDataFrame([aoi_gdf.iloc[i]],  columns=aoi_gdf.columns,crs=aoi_gdf.crs)
range_gdf.to_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\campus_aoi_{}.gpkg'.format(i))

In [5]:
# 拆分 + 分别投影到对应的带号
range_files = [dir+'AOI_gpkg\\campus_aoi_{}.gpkg'.format(i)for i in range(0,length)]
range_gdfs =  BatchRead_gpkg(range_files)

def ProjectionNew(i):
    # 从aoi中读取经度
    lng = range_gdfs[i].loc[0,'lngC'] 
    zone_number = int((lng+ 180) / 6) + 1
    crs = 'EPSG:326{}'.format(zone_number) 

    # 将aoi的经纬度坐标转换为投影坐标
    range_gdfs[i]=range_gdfs[i].to_crs(crs)
    range_gdfs[i]['areaUTM']= range_gdfs[i].area # 计算面积         
    range_gdfs[i].to_file(dir+'AOI_gpkg\\Proj_aoi_{}.gpkg'.format(i), driver='GPKG')

with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(ProjectionNew, i) for i in range(length)]

    # 可选：等待所有任务完成
    for future in futures:
        future.result()  # 这将引发任何在任务中发生的异常

In [49]:
# 如果需要导入
aoi_gdf = gpd.read_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_contextIndex.gpkg') 
range_files = [dir+'AOI_gpkg\\Proj_aoi_{}.gpkg'.format(i)for i in range(0,length)]
range_gdfs =  BatchRead_gpkg(range_files)

In [ ]:
# 更新列表和单个文件的面积值。areaN是albers投影， areaUTM是UTM投影
for i in range(0,1263):
    aoi_gdf.loc[i,'areaUTM'] = range_gdfs[i]['areaUTM'][0]

In [51]:
# 形状指数：采用与正方形的参照 S = P / (4 * sqrt(A))，其中 P 是周长，A 是面积
for i in range(0,1263):
    length_geom = range_gdfs[i].length.sum()
    aoi_gdf.at[i,'length']=length_geom
    aoi_gdf.loc[i,'shapeIndex'] = length_geom/math.sqrt(range_gdfs[i].area)/4
    

In [59]:
aoi_gdf.to_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_contextIndex.gpkg')

### 计算obb的长宽比

In [ ]:
def calculate_length(polygon):
    # 获取多边形边界上的所有点
    coords = list(polygon.exterior.coords)
    #print(len(coords))
    # 初始化最大宽度为0
    max_length = 0
    # 计算所有点对之间的距离
    for i in range(len(coords)-1):
        # 计算两点之间的距离
        point1 = Point(coords[i])
        point2 = Point(coords[i+1])
        # 计算两点之间的距离
        distance = point1.distance(point2)
        #print(distance)
        # 更新最大宽度
        max_length = max(max_length, distance)
    return max_length

In [ ]:
def GetObb(i,range_gdfs):
    gdf = range_gdfs[i]
    obb_gdf = gpd.GeoDataFrame(geometry=[None]*len(gdf),crs=gdf.crs)
    
    # 遍历GeoDataFrame中的每个多边形

    polygon = gdf.geometry[0]

    # 计算每个多边形的最小面积包围矩形
    obb = polygon.minimum_rotated_rectangle

    # 将OBB添加到新的GeoDataFrame中
    obb_gdf.at[0, 'geometry'] = obb
    obb_gdf['campusName']=gdf['campusName']
    
    length_total = obb_gdf.length
    length = calculate_length(obb)
    width = length_total/2-length
    obb_gdf['length'] = length
    obb_gdf['width'] = width
    obb_gdf['ratio'] = length/width
    obb_gdf.to_file(dir+f'AOI_gpkg\\Proj_aoi_oob_{i}.gpkg')
    return obb_gdf
   

In [ ]:
obb_gdfs = [GetObb(i,range_gdfs) for i in range(0,length)]

In [ ]:
# 计算长宽比
aoi_context = gpd.read_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_contextIndex.gpkg')
aoi_context['aspect_ratio'] = [obb_gdfs[i]['ratio'][0]for i in range(0,length)]

aoi_context.to_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_contextIndex.gpkg')

### 计算方向

In [ ]:
# 计算方向
def GetOrient(geom):
    detaX = geom.exterior.coords[0][0] - geom.exterior.coords[1][0]
    detaY = geom.exterior.coords[0][1] - geom.exterior.coords[1][1]
    if detaY < 0:
        detaX = -detaX
        detaY = -detaY
    
    angle = math.atan2(detaY,detaX)
    angle_degrees = math.degrees(angle)
    orient = abs(math.sin(2*angle))
    return orient,angle_degrees

with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交任务到线程池
    futures = [executor.submit(GetOrient, geom=obb_gdfs[i]['geometry'][0]) for i in range(length)]

    # 等待所有任务完成，并按文件编号顺序存储结果
    for i, future in enumerate(futures):
        orient, degree = future.result()
        aoi_context.loc[i, 'orient'] = orient
        aoi_context.loc[i, 'degree'] = degree
        #if i % 10 == 0:
            #print(i, "finished")

In [ ]:
aoi_context.to_file(dir +'AOI_gpkg\\aoi_contextIndex.gpkg')

### 渔网分析


oob是地理坐标系，投影之后在找oob

In [33]:
from shapely.geometry import box
import pandas as pd
from shapely.affinity import rotate

In [27]:
obb_files = [dir+f'AOI_gpkg\\Proj_aoi_oob_{i}.gpkg' for i in range(0,length)]
obb_gdfs  = BatchRead_gpkg(obb_files)


In [35]:
aoi_context = gpd.read_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_contextIndex.gpkg')

#### 对obb划分网格

In [18]:
def GetAngle(angle_degrees):
    if(45<angle_degrees<=90):
        angle = angle_degrees-90
    elif(90<angle_degrees<=135):
        angle = angle_degrees-90
    elif(0<angle_degrees<=45):
        angle = angle_degrees
    elif(135<angle_degrees<=180):
        angle = angle_degrees-180
    return angle

In [46]:
def WidthHeight(obb_gdf,unit):
    geom = obb_gdf.geometry[0]
    minx, miny, maxx, maxy = geom.bounds
    width = maxx - minx
    height = maxy - miny

    nx = round(obb_gdf['width'][0]/unit+0.5)
    ny = round(obb_gdf['length'][0]/unit+0.5)

    cell_width = obb_gdf['width'][0]/nx
    cell_height = obb_gdf['length'][0]/ny
    
    if width < height:
        return nx,ny,cell_width,cell_height
    
    if width > height:
        return ny,nx,cell_height,cell_width

In [47]:
def CreateGrid(index,unit):
    obb_gdf = obb_gdfs[index]
    geom = obb_gdf.geometry[0]

    # 基准点
    origin = geom.centroid
    
    # 判断哪个是x朝向，哪个是y朝向
    nx,ny,cell_width,cell_height = WidthHeight(obb_gdf,unit)
    # 创建网格
    grid = []
    for i in range(nx):
        for j in range(ny):
            x = origin.x + (i -nx/2)* cell_width
            y = origin.y + (j- ny/2) * cell_height
            poly = box(x, y, x + cell_width, y + cell_height)
            grid.append(poly)
            
    # 旋转角度
    angle_degrees=aoi_context['degree'][index] #aoi_context
    
    angle = GetAngle(angle_degrees)
    #print (angle_degrees,angle)
    
    # 旋转网格的多边形
    rotated_grid = []
    for poly in grid:
        rotated_poly = rotate(poly, angle, origin=origin,use_radians=False)
        rotated_grid.append(rotated_poly)

    # 将旋转后的多边形转换为GeoDataFrame
    geoseries = gpd.GeoSeries(rotated_grid)
    rotated_grid_gdf = gpd.GeoDataFrame(geometry=geoseries,crs=obb_gdf.crs)
    
    return rotated_grid_gdf

In [44]:
grid_gdfs=[None]*1623
unit=200 # 200*200网格

In [51]:
grid_gdfs=[None]*1263
unit=200 # 200*200网格
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(CreateGrid, i, unit) for i in range(1263)]
    for i, future in enumerate(futures):
        grid_gdfs[i] = future.result()

#### 裁剪合并处理，获得最终渔网

In [50]:
# 合并较小的部分
def UnionMinArea(cliped,CanMultiPoly=False):
    indexes_to_drop = []

    for index in range(len(cliped)):
        if cliped.loc[index, 'area'] < cliped['area'].max() / 3:
            if index + 1 < len(cliped):  # 确保不会越界
                try:
                    result =  cliped.loc[index,'geometry'].union(cliped.loc[index + 1, 'geometry'])
                    if CanMultiPoly==False and result.geom_type!='Polygon':
                        continue
                    if result is not None and result.is_valid:
                            cliped.loc[index + 1, 'geometry'] = result
                            cliped.loc[index + 1, 'area'] = cliped.loc[index + 1, 'geometry'].area
                            # print(index, cliped.loc[index+1, 'area'])
                            indexes_to_drop.append(index)
                except Exception as e:
                    print(f"An error occurred during union operation for index {index}: {e}")
                
    # 在循环结束后一次性删除所有需要删除的行
    cliped.drop(indexes_to_drop, inplace=True)

In [45]:
range_files = [dir+'AOI_gpkg\\Proj_aoi_{}.gpkg'.format(i)for i in range(0,length)]
range_gdfs =  BatchRead_gpkg(range_files)

In [55]:
# 运行裁剪+合并
def ClipGrid(i):
    cliped = grid_gdfs[i].overlay(range_gdfs[i], how='intersection')[grid_gdfs[i].columns]
    cliped['area'] = cliped.area
    UnionMinArea(cliped)
    cliped_reversed = cliped.iloc[::-1].reset_index(drop=True)
    UnionMinArea(cliped_reversed,True)
    cliped_reversed.to_file(dir+'AOI_gpkg\\0_Grid_{}.gpkg'.format(i))

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(ClipGrid, i) for i in range(12,1263)]
    for i, future in enumerate(futures):
        grid_gdfs[i] = future.result()

## 2. 建筑 

In [8]:
t = 'buildings'
buffer =1.2
dir_buildings= 'E:\\greenland_Campus\\GisDataforChina\\{}_gpkg\\'.format(t)

### 提取校园内建筑 + 融合面 + 面积统计

In [47]:
from shapely import unary_union

In [43]:
def MergeShapefile(gdf, buffer,recover = True):
    
    # 为每个图元创建缓冲区
    gdf['geometry'] = gdf['geometry'].buffer(buffer,cap_style='square')

    # 合并所有缓冲区
    if 'type' in gdf.columns:
        merged_gdf = gdf.dissolve(by='type',aggfunc='first',as_index=False)
    else:
        merged_geom = unary_union(gdf['geometry'])
        # 创建一个新的GeoDataFrame来存储融合后的图元
        merged_gdf = gpd.GeoDataFrame(index=[0], crs=gdf.crs, geometry=[merged_geom])
    
    merged_gdf= merged_gdf.explode()
    merged_gdf.reset_index(drop=True, inplace=True)

    if recover:
        # 缩小融合后的图元
        merged_gdf['geometry'] = merged_gdf['geometry'].buffer(-1*buffer,cap_style='square')
    # 计算面积
    merged_gdf['area'] = merged_gdf.area

    return merged_gdf[merged_gdf['area']>5]
    

In [ ]:
import os
os.environ['PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT'] = '2'  # 设置为2秒

In [85]:
def GetPolygonInside(gdf,range_gdf):
     # 转换坐标系
    #print(gdf.crs)
    selected_gdf = gdf.copy()
    selected_gdf['geometry1'] =selected_gdf['geometry']
    selected_gdf['geometry'] = gdf.geometry.centroid # 使‘geometry’为中点
    selected_gdf = gpd.sjoin(selected_gdf, range_gdf, how='inner', predicate='within') # 提取
    selected_gdf['geometry']=selected_gdf['geometry1'] 
    selected_gdf.drop(columns=['geometry1'],inplace=True) #恢复‘geometry’
    return selected_gdf

In [86]:
building_files = [dir_buildings+'{}_{}.gpkg'.format(t,i) for i in range(0,length)]
building_gdfs= BatchRead_gpkg(building_files) #读取建筑文件

In [87]:
# run
def CalFA(i):

    range_gdf =range_gdfs[i] 
    print(range_gdf.crs)
    # 提取校园内的建筑
    gdf = building_gdfs[i].to_crs(range_gdf.crs)
    selected_gdf = GetPolygonInside(gdf,range_gdf) 

    # 合并校园内邻近的矢量
    selected_gdf = MergeShapefile(selected_gdf,buffer) 

    # 导出文件
    selected_gdf.to_file(dir_buildings+'0_inside_{}_{}.gpkg'.format(t,i), driver='GPKG')
    #print('finish',i)
    # 统计面积
    return selected_gdf.area.sum() 


In [ ]:
FA = {}
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(CalFA, i) for i in range(0,1263)]
    for i, future in enumerate(futures):
        FA[i] = future.result()

In [ ]:
aoi_buildings=gpd.read_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_buildingIndex.gpkg')

In [92]:
aoi_buildings['FA']=FA
aoi_buildings['density_BD']=aoi_buildings['FA']/aoi_buildings['areaUTM']

In [97]:
aoi_buildings.to_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_buildingIndex.gpkg', driver='GPKG')


### 计算指标

In [9]:
# 批量文件位置
gpkg_files = [dir_buildings+'0_inside_{}_{}.gpkg'.format(t,i) for i in range(0,length)]
# 读进内存
gdfs = BatchRead_gpkg(gpkg_files)

#### 多边形最近邻距离

In [20]:
def NN(gdf):
    buildings_gdf = gdf
    if buildings_gdf.area.sum()==0:
        return 0,0

    # 初始化一个列表来存储最近邻距离
    nearest_distances = []

    # 对于每个建筑物，找出最近的邻居并计算距离
    for index, building in buildings_gdf.iterrows():
        # 找出除自身外最近的建筑物
        possible_neighbors = buildings_gdf.index.difference([index])
        distances = buildings_gdf.loc[possible_neighbors].distance(building['geometry'])
        nearest_neighbor_distance = distances.min()
        
        # 将最近邻距离添加到列表中
        nearest_distances.append(nearest_neighbor_distance)

    # 计算平均最近邻距离
    nn_avr = round(sum(nearest_distances) / len(nearest_distances),4)
    nn_std = np.std(nearest_distances, ddof=0)
    return nn_avr,nn_std

In [ ]:
avrs = {}
stds = {}
with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交任务到线程池
    futures = [executor.submit(NN, gdf) for gdf in gdfs]

    # 等待所有任务完成，并按文件编号顺序存储结果
    for i, future in enumerate(futures):
        avr, std = future.result()
        avrs[i] = avr
        stds[i] = std

        if i % 10 == 0:
            print(i, "finished")

In [101]:
aoi_buildings['NN_avr1'] = avrs
aoi_buildings['NN_std1'] = stds

#### 中心点最近邻指数

In [102]:
def NNI(gdf,area):
    
    if gdf.area.sum()==0:
        return 0

    # 初始化一个列表来存储最近邻距离
    nearest_distances = []

    gdf['centroid'] = gdf.geometry.centroid

    # 对于每个建筑物，找出最近的邻居并计算距离
    for index, geom in gdf.iterrows():
        # 找出除自身外最近的
        possible_neighbors = gdf.index.difference([index])
        distances = gdf.loc[possible_neighbors,'centroid'].distance(geom['centroid'])
        nearest_neighbor_distance = distances.min()
        
        # 将最近邻距离添加到列表中
        nearest_distances.append(nearest_neighbor_distance)

    # 计算平均最近邻指数
    dd = 0.5/math.sqrt(gdf.shape[0]/area)
    nni = np.mean(nearest_distances)/dd

    return nni

In [ ]:
nnis= {}
with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交任务到线程池
    futures = [executor.submit(NNI, gdfs[i],aoi_buildings.loc[i,'areaUTM']) for i in range(length)]

    # 等待所有任务完成，并按文件编号顺序存储结果
    for i, future in enumerate(futures):
        nnis[i]= future.result()
        if i % 100 == 1:
            print(i, "finished")


In [104]:
aoi_buildings['NNI1'] = nnis

In [105]:
aoi_buildings.to_file(dir+'AOI_gpkg\\aoi_buildingIndex.gpkg', driver='GPKG')

### 分布均衡性

In [36]:
aoi_buildings = gpd.read_file('E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\aoi_buildingIndex.gpkg')

In [106]:
def PolygonEveness(grid_gdf,poly_gdf,column_name):

    result_gdf = grid_gdf
    result_gdf[column_name] = 0.0  # 初始化道路长度字段

    for index, net_polygon in grid_gdf.iterrows():
        net_geometry = net_polygon['geometry']
        
        # 遍历道路的每个线要素
        for index01, row in poly_gdf.iterrows():
            geometry = row['geometry']
            
            # 检查是否与渔网多边形相交
            if net_geometry.intersects(geometry):
                # 计算交集部分
                intersection = net_geometry.intersection(geometry)
                
                # 如果交集是 LineString，计算其长度并累加到结果中
                result_gdf.at[index, column_name] += intersection.area

    # 删除过小的部分
    #result_gdf=result_gdf[result_gdf['area']>1/3*result_gdf['area'].mean()]
    # 标准差
    mean = result_gdf[column_name].mean()
    std = result_gdf[column_name].std(ddof=1)
    if mean !=0:
        eveness = std/mean
    else:
        eveness = 0
    return eveness

In [107]:
print(PolygonEveness(grid_gdfs[0],gdfs[0],'building_area'))

0.5845720757624524


In [8]:
grid_files = [dir+'AOI_gpkg\\0_Grid_{}.gpkg'.format(i)for i in range(0,length)]
grid_gdfs = BatchRead_gpkg(grid_files)

In [108]:
results = {}
for i in range(length):
    results[i]=PolygonEveness(grid_gdfs[i],gdfs[i],'building_area')

In [109]:
aoi_buildings['buildings_eveness'] = results


In [110]:
aoi_buildings.drop(columns=['NNI','NN_avr','NN_std'], inplace=True)


,density_BD,areaUTM,campusName,FA,geometry,buildings_eveness,NN_avr1,NN_std1,NNI1
0,0.286141,3.891525e+05,海南热带海洋学院,111352.391399,"POLYGON ((109.53224 18.31567, 109.53225 18.315...",0.584572,16.8589,16.900711,1.224885
1,0.099290,1.088187e+06,三亚学院(北校区),108045.906314,"POLYGON ((109.55245 18.34337, 109.55246 18.343...",0.954970,14.4720,10.369685,0.952668
2,0.007289,4.896646e+05,海南大学(观澜湖校区),3569.027558,"POLYGON ((110.33655 19.92597, 110.33656 19.925...",1.545056,91.2387,110.730790,1.133506
3,0.181722,8.629453e+05,海口经济学院,156815.887941,"POLYGON ((110.48029 19.97086, 110.48022 19.970...",0.607524,22.8205,11.714278,1.254938
4,0.094855,6.805635e+05,琼台师范学院(桂林洋校区),64554.718650,"POLYGON ((110.51748 19.98019, 110.51749 19.980...",0.885825,22.5384,20.143326,1.039606


In [111]:
aoi_buildings.rename(columns={'NN_avr1':'NN_avr',"NN_std1":"NN_std","NNI1":"NNI"},inplace=True)
aoi_buildings.head()

,density_BD,areaUTM,campusName,FA,geometry,buildings_eveness,NN_avr,NN_std,NNI
0,0.286141,3.891525e+05,海南热带海洋学院,111352.391399,"POLYGON ((109.53224 18.31567, 109.53225 18.315...",0.584572,16.8589,16.900711,1.224885
1,0.099290,1.088187e+06,三亚学院(北校区),108045.906314,"POLYGON ((109.55245 18.34337, 109.55246 18.343...",0.954970,14.4720,10.369685,0.952668
2,0.007289,4.896646e+05,海南大学(观澜湖校区),3569.027558,"POLYGON ((110.33655 19.92597, 110.33656 19.925...",1.545056,91.2387,110.730790,1.133506
3,0.181722,8.629453e+05,海口经济学院,156815.887941,"POLYGON ((110.48029 19.97086, 110.48022 19.970...",0.607524,22.8205,11.714278,1.254938
4,0.094855,6.805635e+05,琼台师范学院(桂林洋校区),64554.718650,"POLYGON ((110.51748 19.98019, 110.51749 19.980...",0.885825,22.5384,20.143326,1.039606


In [112]:
aoi_buildings.to_file(dir+'AOI_gpkg\\aoi_buildingIndex.gpkg', driver='GPKG')

### 平均朝向

In [12]:
# 计算方向
def GetBuildingOrient(geom):
    points = [Point(geom.exterior.coords[i]) for i in range(0,3)]
    detaX = points[0].x - points[1].x
    detaY = points[0].y - points[1].y
    # 第一条边是不是长边
    edge1 = points[0].distance(points[1])
    edge2 = points[1].distance(points[2])
    if edge1<edge2:
        a = detaY
        detaY=detaX
        detaX=a
    # -90~90°
    if detaX < 0:
        detaX = -detaX
        detaY = -detaY
    if detaX==0:
        angle_degrees = 90
    else:
        angle = math.atan2(detaY,detaX)
        angle_degrees = math.degrees(angle)
    if angle_degrees>45:
        angle_degrees = angle_degrees-90
    elif angle_degrees<-45:
        angle_degrees = angle_degrees+90
    return angle_degrees

In [16]:
def BuildingOrientIndex(gdf):
    angles=[]
    if gdf['area'].sum()==0:
        return 0,0,0,0
    for polygon in gdf.geometry:
        obb = polygon.minimum_rotated_rectangle
        # 求长边的角度
        angle = GetBuildingOrient(obb)
        angles.append(angle) # 限定在 -45~45
        #weighted_angles.append(angle*polygon.area)

    gdf['angle']=angles
    gdf['weighted_angle']=angles*gdf['area']

    # 求平均值、标准差
    avr = gdf['angle'].mean()
    std = gdf['angle'].std(ddof=0)

    # 加权平均、加权标准差
    avr_weighted = gdf['weighted_angle'].sum()/gdf['area'].sum()
    std_weighted = ((gdf['angle']-avr_weighted)**2*gdf['area']).sum()/gdf['area'].sum()
    std_weighted = math.sqrt(std_weighted)

    return avr,std,avr_weighted,std_weighted

results = {}
with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(BuildingOrientIndex, gdf) for gdf in gdfs]

    # 可选：等待所有任务完成
    for i,future in enumerate(futures):
        results[i]=future.result()

In [33]:
columnNames=['buildings_avr','buildings_std','buildings_avr_weighted','buildings_std_weighted']

In [ ]:
new_columns_data = {col: [values[i] for values in results.values()] for i, col in enumerate(columnNames)}

In [45]:
for columnName in columnNames:
    aoi_buildings[columnName] = new_columns_data[columnName]

In [47]:
aoi_buildings.to_file(dir+'AOI_gpkg\\aoi_buildingIndex.gpkg', driver='GPKG')

## 3. 绿地

In [74]:
t = 'green'
dir_green= 'E:\\greenland_Campus\\GisDataforChina\\{}_gpkg\\'.format(t)

In [ ]:
range_files = ['E:\\greenland_Campus\\GisDataforChina\\AOI_gpkg\\Proj_aoi_{}.gpkg'.format(i)for i in range(0,length)]
range_gdfs =  BatchRead_gpkg(range_files)

In [ ]:
# 这边因为之前没有投影对，重新投影处理，分别投影到对应带号的投影坐标系
# 如果是对的直接读取投影后的文件即可
gpkg_files = [dir_green+'{}_{}.gpkg'.format(t,i) for i in range(0,length)]
# 读进内存
gdfs = BatchRead_gpkg(gpkg_files)

def ProjectionNew(i):
    # 从aoi中读取经度
    lng = range_gdfs[i].loc[0,'lngC'] 
    zone_number = int((lng+ 180) / 6) + 1
    crs = 'EPSG:326{}'.format(zone_number) 

    # 将矢量数据转换为投影坐标
    gdfs[i]=gdfs[i].to_crs(crs)
    gdfs[i].to_file(dir_green+'Proj_{}_{}.gpkg'.format(t,i), driver='GPKG')

with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(ProjectionNew, i) for i in range(length)]

    # 可选：等待所有任务完成
    for future in futures:
        future.result()  # 这将引发任何在任务中发生的异常

### 提取内部的绿地和水域，相交的斑块实施裁剪

In [ ]:
gpkg_files = [dir_green+'Proj_{}_{}.gpkg'.format(t,i) for i in range(0,length)]
# 读进内存
gdfs = BatchRead_gpkg(gpkg_files)

In [ ]:
gdfs_inside =  [None] * 1623
def GetGreenInside(i):
    # 提取内部的绿地和水域 (空的怎么办)
    selected_gdf =gdfs[i].overlay(range_gdfs[i], how='intersection')[gdfs[i].columns]
    # 合并校园内邻近的矢量
    selected_gdf = MergeShapefile(selected_gdf,1,recover=False) # 合并后的文件位置

    # 计算面积
    selected_gdf['area']=selected_gdf.area
    # 导出文件
    selected_gdf.to_file(dir_green+'1_inside_green_{}.gpkg'.format(i), driver='GPKG')

    # 将结果存储在内存
    gdfs_inside[i]=selected_gdf

    if i%100==1:
        print('finish',i)
with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(GetGreenInside, i) for i in range(0,1263)]

    # 可选：等待所有任务完成
    for future in futures:
        future.result()  # 这将引发任何在任务中发生的异常

### 计算指标

In [88]:
aoi_green= gpd.read_file(dir+'AOI_gpkg\\aoi_greenIndex.gpkg')

In [75]:
inside_gpkg_files = [dir_green+'1_inside_{}_{}.gpkg'.format(t,i) for i in range(0,length)]
# 读进内存
gdfs_inside = BatchRead_gpkg(inside_gpkg_files)

In [ ]:
column_names = ['river_ratio', 'green_ratio', 'green_EdgeDensity',
                'green_LPI','green_Fragmentation','green_shapeIndex',
                'green_shapeDiversity','green_areaDiversity','green_dist_avr','green_dist_std']
for column_name in column_names:
    aoi_green[column_name] = None

In [ ]:
for i in range(0,1263):
    gdfs_inside[i]['length'] = gdfs_inside[i].geometry.length
    series  = [math.sqrt(np.pi*gdfs_inside[i].loc[j,'area'])for j in range(len(gdfs_inside[i]))]
    gdfs_inside[i]['shapeIndex']= gdfs_inside[i]['length']/series/2

In [ ]:
def CalGreen(i):
    totalArea = aoi_green.loc[i,'areaUTM']
    gdf =  gdfs_inside[i].copy()
    area = gdf['area'].sum()
    
    if area==0:
        return i,None
    
    result = {}
    
    # 河流比例
    river_area = gdf[gdf['type'] == 'river']['area'].sum()
    result['river_ratio'] = river_area/totalArea
    # 绿地比例
    greenArea = gdf[gdf['type'] == 'green']['area'].sum()
    result['green_ratio'] = greenArea/totalArea
    # 景观边缘密度，这里用整个校园的面积
    result['green_EdgeDensity']= gdf['length'].sum()/totalArea

    # 破碎度：最大斑块指数
    result['green_LPI'] = gdf['area'].max()/area
    # 破碎度：斑块密度，数量/绿地面积
    result['green_Fragmentation']= (len(gdf)-1)/area
    # 平均形状指数
    result['green_shapeIndex'] = gdf['shapeIndex'].mean()

    # 差异度：形状标准差
    result['green_shapeDiversity'] = gdf['shapeIndex'].std()
    # 差异度：面积,标准差/平均面积
    result['green_areaDiversity'] = gdf['area'].std()/area
    # 种类？

    # 均匀性xx
    result['green_dist_avr'], result['green_dist_std'] = NN(gdf)
    # green_NNI = NNI(gdf,totalArea)

    return i, result

with ProcessPoolExecutor() as executor:
        futures = [executor.submit(CalGreen, i) for i in range(0,1263)]
        results = []
        for future in as_completed(futures):
            result = future.result()
            if result[1] is not None:
                results[result[0]] = result[1]

for i, data in results.items():
        for key, value in data.items():
            aoi_green.at[i, key] = value



### 均衡性

In [87]:
results = {}
for i in range(1263):
    results[i] = PolygonEveness(grid_gdfs[i],gdfs_inside[i],'green_area')
aoi_green['green_eveness'] = results
aoi_green.to_file(dir+'AOI_gpkg\\aoi_greenIndex.gpkg', driver='GPKG')

In [ ]:
# 多样性、服务水平(和建筑的关系)

## 4. 道路

In [5]:
t = 'road'
dir_road= 'E:\\greenland_Campus\\GisDataforChina\\{}_gpkg\\'.format(t)

利用qgis的空间连接（按位置连接属性）把属性添加上

In [ ]:
processing.run("native:joinattributesbylocation", {'INPUT':'E:/greenland_Campus/GisDataforChina/road_gpkg/CenterLine_road_0.gpkg','PREDICATE':[0],'JOIN':'E:/greenland_Campus/GisDataforChina/road_gpkg/Proj_road_0.gpkg','JOIN_FIELDS':[],'METHOD':2,'DISCARD_NONMATCHING':False,'PREFIX':'','OUTPUT':'ogr:dbname=\'E:/greenland_Campus/GisDataforChina/road_gpkg/NCenterline_0.gpkg\' table="road" (geom)'})

In [ ]:
gpkg_files = [dir_road+'1_centerLine_{}_{}.gpkg'.format(t,i) for i in range(0,length)]
# 读进内存
gdfs = BatchRead_gpkg(gpkg_files)

In [ ]:
range_files = [dir+'AOI_gpkg\\Proj_aoi_{}.gpkg'.format(i)for i in range(0,length)]
range_gdfs =  BatchRead_gpkg(range_files)

In [ ]:
# 这边因为之前没有投影对，重新投影处理，分别投影到对应带号的投影坐标系
# 如果是对的直接读取投影后的文件即可

def ProjectionNew(i):
    
    # 将矢量数据转换为投影坐标
    gdfs[i]=gdfs[i].to_crs(range_gdfs[i].crs)
    gdfs[i].to_file(gpkg_files[i], driver='GPKG')

with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(ProjectionNew, i) for i in range(length)]

    # 可选：等待所有任务完成
    for future in futures:
        future.result()  # 这将引发任何在任务中发生的异常

In [ ]:
# 裁剪 内部
gdfs_inside =  [None] * 1623
def GetRoadInside(i):
    # 提取内部的道路 (有空的)
    selected_gdf =gdfs[i].overlay(range_gdfs[i], how='intersection')[gdfs[i].columns]

    # 计算len
    selected_gdf['length']=selected_gdf.length
    selected_gdf.dissolve(by='DN', aggfunc='sum')

    selected_gdf= selected_gdf[selected_gdf['length']>5]
    selected_gdf.explode()
    # 导出文件
    selected_gdf.to_file(gpkg_files[i].replace('1_centerLine','inside'), driver='GPKG')

    # 将结果存储在内存
    gdfs_inside[i]=selected_gdf

    if i%100==1:
        print('finish',i)
with ThreadPoolExecutor(max_workers=10) as executor: 
    # 提交所有任务
    futures = [executor.submit(GetRoadInside, i) for i in range(0,1263)]

    # 可选：等待所有任务完成
    for future in futures:
        future.result()  # 这将引发任何在任务中发生的异常

### index

In [ ]:
import networkx as nx

In [6]:
gpkg_files = [dir_road+'inside_{}_{}.gpkg'.format(t,i) for i in range(0,length)]
# 读进内存
gdfs_inside = BatchRead_gpkg(gpkg_files)

In [7]:
aoi_road = gpd.read_file(dir+'AOI_gpkg\\aoi_roadIndex.gpkg')

In [ ]:
def ToGraph(gdf):
    G = nx.Graph()

    # 遍历GeoDataFrame中的每个线要素（道路）
    for index,row in gdf.iterrows():
        geom = row.geometry
        coords = list(geom.coords)  # 获取线的所有顶点坐标
        
        # 提取线的端点
        if len(coords) > 0:
            start_point = coords[0]
            end_point = coords[-1]
            
            # 添加端点到图中
            if not G.has_node(start_point):
                G.add_node(start_point)
            if not G.has_node(end_point):
                G.add_node(end_point)
            
            # 添加边到图中
            G.add_edge(start_point, end_point,length=geom.length,weight=row['DN'])
        
        
    return G

In [ ]:
def CalRoadIndex(i):
    G = ToGraph(gdfs_inside[i].explode())
    result = {}
    # basic index
    result['intersection_count'] = sum(1 for node, degree in G.degree() if degree > 1)
    result['segment_counts'] = G.number_of_edges()
    result['total_length'] = sum(G[u][v]['length'] for u, v in G.edges())
    result['total_length_weighted'] = sum(G[u][v]['length']*G[u][v]['weight'] for u, v in G.edges())

    #nx.write_gpickle(G, dir_road+'G_inside_{}.gpickle')
    nx.write_graphml(G, dir_road+f'G_inside_{i}.gpickle.graphml')  
    return i,result
with ThreadPoolExecutor() as executor:
        futures = [executor.submit(CalRoadIndex, i) for i in range(1263)]
        results = {}
        for future in as_completed(futures):
            result = future.result()
            if result[1] is not None:
                results[result[0]] = result[1]

for i, data in results.items():
        for key, value in data.items():
            aoi_road.at[i, key] = value


In [ ]:
aoi_road['road_density'] = aoi_road['total_length_weighted'] / aoi_road['areaUTM']
aoi_road['avr_length']= aoi_road['total_length'] / aoi_road['segment_counts']
aoi_road['intersection_Density']= aoi_road['intersection_count'] / aoi_road['areaUTM']
aoi_road['complexsity'] = aoi_road['total_length'] * aoi_road['intersection_count']+1

In [ ]:
aoi_road.to_file(dir+'AOI_gpkg\\aoi_roadIndex.gpkg')

In [ ]:
# first_edge = list(G.edges(data=False))[0]

In [ ]:
# 绘图
pos = {node: node for node in G.nodes()}
nx.draw(G, pos, with_labels=False,node_size=20)
plt.show()

### NNI

In [ ]:
from shapely.geometry import LineString, Point

In [ ]:
def road_NNI(i):
   
    gdf = gdfs_inside[i].explode()
    area = aoi_road.loc[i,'areaUTM']
    if gdf.shape[0]==0:
        return i,0
    # 初始化一个列表来存储最近邻距离
    nearest_distances = []

    gdf['centroid'] = gdf.geometry.centroid

    # 对于每个建筑物，找出最近的邻居并计算距离
    for index, geom in gdf.iterrows():
        # 找出除自身外最近的
        possible_neighbors = gdf.index.difference([index])
        distances = gdf.loc[possible_neighbors,'centroid'].distance(geom['centroid'])
        nearest_neighbor_distance = distances.min()
        
        # 将最近邻距离添加到列表中
        nearest_distances.append(nearest_neighbor_distance)

    # 计算平均最近邻指数
    dd = 0.5/math.sqrt(gdf.shape[0]/area)
    nni = np.mean(nearest_distances)/dd

    return i,nni

In [ ]:
with ThreadPoolExecutor() as executor:
        futures = [executor.submit(road_NNI, i) for i in range(602,1263)]
        results = {}
        for future in futures:
            result = future.result()
            aoi_road.at[result[0], 'road_nni'] = result[1]

In [ ]:
aoi_road.to_file(dir+'AOI_gpkg\\aoi_roadIndex.gpkg')

### 均衡性

In [56]:
grid_files = [dir+'AOI_gpkg\\0_Grid_{}.gpkg'.format(i)for i in range(0,length)]
grid_gdfs = BatchRead_gpkg(grid_files)

In [66]:
# 遍历渔网的每个多边形
def RoadEveness(grid,road):
    # grid = grid_gdfs[i]
    # road = gdfs_inside[i]
    result_gdf = grid
    result_gdf['total_road_length'] = 0.0  # 初始化道路长度字段

    for index, net_polygon in grid.iterrows():
        net_geometry = net_polygon['geometry']
        
        # 遍历道路的每个线要素
        for road_index, road_line in road.iterrows():
            road_geometry = road_line['geometry']
            
            # 检查道路是否与渔网多边形相交
            if net_geometry.intersects(road_geometry):
                # 计算交集部分
                intersection = net_geometry.intersection(road_geometry)
                
                # 如果交集是 LineString，计算其长度并累加到结果中
                if intersection.geom_type == 'LineString':
                    result_gdf.at[index, 'total_road_length'] += intersection.length
    # 删除过小的部分
    result_gdf=result_gdf[result_gdf['area']>1/3*result_gdf['area'].mean()]
    # 标准差
    mean = result_gdf.total_road_length.mean()
    std = result_gdf.total_road_length.std(ddof=1)
    eveness = std/mean
    return eveness

In [ ]:
results = {}
for i in range(1263):
    results[i] = RoadEveness(grid_gdfs[i],gdfs_inside[i])

In [67]:
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(RoadEveness, grid_gdfs[i],gdfs_inside[i]) for i in range(1263)]
    results = {}
    for i,future in enumerate(futures):
        result = future.result()
        results[i] = result

In [71]:

aoi_road['road_eveness'] = results
aoi_road

In [73]:
aoi_road.to_file(dir+'AOI_gpkg\\aoi_roadIndex.gpkg')